### Watson creds & Model parameters

In [89]:
pip install ibm-watson

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from ibm_watsonx_ai import APIClient, Credentials
import getpass

credentials = Credentials(
    url="https://eu-gb.ml.cloud.ibm.com",
    api_key= input("Enter your IBM CLoud API Key:")
)

In [91]:
model_id = "meta-llama/llama-4-maverick-17b-128e-instruct-fp8"

In [92]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

In [93]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")

In [94]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
	model_id = model_id,
	params = parameters,
	credentials = credentials,
	project_id = project_id,
	space_id = space_id
	)

### Web page fetching & content cleanup function (extract text from html page)

In [ ]:
import requests
import trafilatura

url = input("Enter the Wikipedia URL:")

downloaded = trafilatura.fetch_url(url)

if downloaded:
    extracted_text = trafilatura.extract(downloaded, include_comments=False, include_tables=False)
    #print(extracted_text)
else:
    print("Failed to fetch the content.")

In [ ]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, RelationsOptions

# --- Neo4j Imports ---
from neo4j import GraphDatabase

# --- Configuration ---
# Replace with your IBM Cloud API Key for Natural Language Understanding
API_KEY = input("Enter your IBM Watson NLU API Key: ")

# Prompt for NLU service URL
SERVICE_URL = input("Enter your IBM Watson NLU Service URL: ")


text_to_analyze = extracted_text



# --- Neo4j Configuration ---
NEO4J_URI = input("Enter your Neo4j URI: ")
NEO4J_USERNAME = input("Enter your Neo4j username: ")
NEO4J_PASSWORD = getpass.getpass("Enter your Neo4j password: ")

# --- Initialize NLU Service ---
try:
    authenticator = IAMAuthenticator(API_KEY)
    natural_language_understanding = NaturalLanguageUnderstandingV1(
        version='2022-04-07',
        authenticator=authenticator
    )
    natural_language_understanding.set_service_url(SERVICE_URL)

    print("Watson Natural Language Understanding service initialized successfully.\n")

    # --- Define Features for Analysis ---
    features = Features(
        relations=RelationsOptions()
    )

    # --- Analyze the Text ---
    print("Analyzing text for relations...\n")
    response = natural_language_understanding.analyze(
        text=text_to_analyze, #text
        features=features #relations
    ).get_result()

    # --- Initialize Neo4j Driver ---
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    print("Connected to Neo4j successfully.\n")

    # --- Process and Print Results & Send to Neo4j ---
    if 'relations' in response and len(response['relations']) > 0:
        print("Extracted Relations:")
        with driver.session() as session:
            for relation in response['relations']:
                rel_type = relation.get('type')
                rel_score = relation.get('score')
                rel_sentence = relation.get('sentence')


                arguments = relation.get('arguments', [])
                if len(arguments) >= 2: # Ensure there are at least two arguments for a relation
                    arg1 = arguments[0]
                    arg2 = arguments[1]

                    arg1_text = arg1.get('text')
                    arg1_type = arg1.get('entities', [{}])[0].get('type') if arg1.get('entities') else "Unknown"

                    arg2_text = arg2.get('text')
                    arg2_type = arg2.get('entities', [{}])[0].get('type') if arg2.get('entities') else "Unknown"


                    # Create or merge nodes for entities
                    # Use MERGE to avoid creating duplicate nodes if they already exist
                    create_node_query = """
                    MERGE (a:{arg1_type} {{name: '{arg1_text}'}})
                    MERGE (b:{arg2_type} {{name: '{arg2_text}'}})
                    CREATE (a)-[r:{rel_type} {{score: {rel_score}, sentence: '{rel_sentence}'}}]->(b)
                    RETURN a, b, r
                    """
                    # Sanitize inputs for Cypher
                    sanitized_arg1_text = arg1_text.replace("'", "\\'")
                    sanitized_arg2_text = arg2_text.replace("'", "\\'")
                    sanitized_rel_sentence = rel_sentence.replace("'", "\\'")

                    cypher_query = f"""
                    MERGE (a:{arg1_type} {{name: '{sanitized_arg1_text}'}})
                    MERGE (b:{arg2_type} {{name: '{sanitized_arg2_text}'}})
                    CREATE (a)-[r:{rel_type} {{score: {rel_score}, sentence: '{sanitized_rel_sentence}'}}]->(b)
                    RETURN a, b, r
                    """
                    try:
                        session.run(cypher_query)
                        print(f"  Successfully sent relation '{rel_type}' between '{arg1_text}' and '{arg2_text}' to Neo4j.")
                    except Exception as neo4j_e:
                        print(f"  Error sending to Neo4j: {neo4j_e}")
                else:
                    print("    Insufficient arguments for a relation.")

                print("-" * 30)
    else:
        print("No relations found in the provided text.")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure your API_KEY and SERVICE_URL are correct and you have access to the NLU service, and your Neo4j credentials are valid.")
finally:
    # Close the Neo4j driver connection
    if 'driver' in locals() and driver:
        driver.close()
        print("Neo4j connection closed.")

Watson Natural Language Understanding service initialized successfully.

Analyzing text for relations...

Connected to Neo4j successfully.

Extracted Relations:
  Successfully sent relation 'managerOf' between 'King' and 'Saudi Arabia' to Neo4j.
------------------------------
  Successfully sent relation 'residesIn' between 'viceroy' and 'region' to Neo4j.
------------------------------
  Successfully sent relation 'locatedAt' between 'They' and 'Blair House' to Neo4j.
------------------------------
  Successfully sent relation 'employedBy' between 'guest' and 'government' to Neo4j.
------------------------------
  Successfully sent relation 'ownerOf' between 'guest' and 'house' to Neo4j.
------------------------------
  Successfully sent relation 'locatedAt' between 'guest' and 'Blair House' to Neo4j.
------------------------------
  Successfully sent relation 'locatedAt' between 'house' and 'Blair House' to Neo4j.
------------------------------
  Successfully sent relation 'locatedAt

In [102]:
#result = session.run(query)

SessionError: Session closed